Endpoints

In [1]:
%pip install flask

%pip install flask-cors

%pip install flask_sqlalchemy

%pip install routes

%pip install requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sqlite3

# Connect to the uploaded SQLite database
db_path = "../models/cinemind.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Retrieve the table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Get schema details for each table
schema_info = {}
for table in tables:
    table_name = table[0]
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema_info[table_name] = cursor.fetchall()

# Close connection
conn.close()

# Display schema details
schema_info

{'Movies': [(0, 'id', 'INTEGER', 0, None, 1),
  (1, 'title', 'TEXT', 1, None, 0),
  (2, 'original_title', 'TEXT', 0, None, 0),
  (3, 'overview', 'TEXT', 0, None, 0),
  (4, 'budget', 'REAL', 0, None, 0),
  (5, 'revenue', 'REAL', 0, None, 0),
  (6, 'release_date', 'TEXT', 0, None, 0),
  (7, 'runtime', 'INTEGER', 0, None, 0),
  (8, 'status', 'TEXT', 0, None, 0),
  (9, 'tagline', 'TEXT', 0, None, 0),
  (10, 'popularity', 'REAL', 0, None, 0),
  (11, 'vote_average', 'REAL', 0, None, 0),
  (12, 'vote_count', 'INTEGER', 0, None, 0),
  (13, 'original_language', 'TEXT', 0, None, 0),
  (14, 'homepage', 'TEXT', 0, None, 0)],
 'sqlite_sequence': [(0, 'name', '', 0, None, 0), (1, 'seq', '', 0, None, 0)],
 'Genres': [(0, 'genre_id', 'INTEGER', 0, None, 1),
  (1, 'genre_name', 'TEXT', 0, None, 0)],
 'Movie_Genre': [(0, 'movie_id', 'INTEGER', 1, None, 1),
  (1, 'genre_id', 'INTEGER', 1, None, 2)],
 'Production_Countries': [(0, 'country_id', 'INTEGER', 0, None, 1),
  (1, 'country_name', 'TEXT', 0, None,

testing to see if sqlite data had imported from file correctly (all data is shown)

All classes written in python format

In [3]:
class Cast:
    def __init__(self, actor_id, name, gender):
        self.actor_id = actor_id
        self.name = name
        self.gender = gender
        
class Genres:
    def __init__(self, genre_id, genre_name):
        self.genre_id = genre_id
        self.genre_name = genre_name

class Keywords:
    def __init__(self, keyword_id, keyword_name):
        self.keyword_id = keyword_id
        self.keyword_name = keyword_name

class Movie_Genre:
    def __init__(self, movie_id, genre_id):
        self.movie_id = movie_id
        self.genre_id = genre_id

class Movie_Keywords:
    def __init__(self, movie_id, keyword_id):
        self.movie_id = movie_id
        self.keyword_id = keyword_id

class Movie_Production_Countries:
    def __init__(self, movie_id, country_id):
        self.movie_id = movie_id
        self.country_id = country_id

class Movie_Spoken_Languages:
    def __init__(self, movie_id, language_id):
        self.movie_id = movie_id
        self.language_id = language_id

class Movies:
    def __init__(self, id, title, original_title, overview, budget, revenue, release_date, runtime, status, tagline, popularity, vote_average, vote_count, original_language, homepage):
        self.id = id
        self.title = title
        self.original_title = original_title
        self.overview = overview
        self.budget = budget
        self.revenue = revenue
        self.release_date = release_date
        self.runtime = runtime
        self.status = status
        self.tagline = tagline
        self.popularity = popularity
        self.vote_average = vote_average
        self.vote_count = vote_count
        self.original_language = original_language
        self.homepage = homepage

class Movies_Cast:
    def __init__(self, movie_id, actor_id):
        self.movie_id = movie_id
        self.actor_id = actor_id

class Production_Countries:
    def __init__(self, country_id, country_name, iso_code):
        self.country_id = country_id
        self.country_name = country_name
        self.iso_code = iso_code

class Spoken_Languages:
    def __init__(self, language_id, language_name, iso_code):
        self.language_id = language_id
        self.language_name = language_name
        self.iso_code = iso_code

Cell below has all classes and endpoints made however, SystemExit 1 error - fixed, Internal Server Error appears on page - key error on "movies" etc.

In [4]:
from flask import Flask, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask import request

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///cinemind.db'  # Use appropriate database URI
db = SQLAlchemy(app)

class Cast(db.Model):
    actor_id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100))
    gender = db.Column(db.String(10))

class Genres(db.Model):
    genre_id = db.Column(db.Integer, primary_key=True)
    genre_name = db.Column(db.String(100))

class Keywords(db.Model):
    keyword_id = db.Column(db.Integer, primary_key=True)
    keyword_name = db.Column(db.String(100))

class Movie_Genre(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movies.id'), primary_key=True)
    genre_id = db.Column(db.Integer, db.ForeignKey('genres.genre_id'), primary_key=True)
    movie = db.relationship('Movies', back_populates='genres')
    genre = db.relationship('Genres', back_populates='movies')

class Movie_Keywords(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movies.id'), primary_key=True)
    keyword_id = db.Column(db.Integer, db.ForeignKey('keywords.keyword_id'), primary_key=True)
    movie = db.relationship('Movies', back_populates='keywords')
    keyword = db.relationship('Keywords', back_populates='movies')

class Movie_Production_Countries(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movies.id'), primary_key=True)
    country_id = db.Column(db.Integer, db.ForeignKey('production_countries.country_id'), primary_key=True)
    movie = db.relationship('Movies', back_populates='production_countries')
    country = db.relationship('Production_Countries', back_populates='movies')

class Movie_Spoken_Languages(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movies.id'), primary_key=True)
    language_id = db.Column(db.Integer, db.ForeignKey('spoken_languages.language_id'), primary_key=True)
    movie = db.relationship('Movies', back_populates='spoken_languages')
    language = db.relationship('Spoken_Languages', back_populates='movies')

class Movies(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200))
    original_title = db.Column(db.String(200))
    overview = db.Column(db.Text)
    budget = db.Column(db.Integer)
    revenue = db.Column(db.Integer)
    release_date = db.Column(db.String(20))
    runtime = db.Column(db.Integer)
    status = db.Column(db.String(50))
    tagline = db.Column(db.String(200))
    popularity = db.Column(db.Float)
    vote_average = db.Column(db.Float)
    vote_count = db.Column(db.Integer)
    original_language = db.Column(db.String(50))
    homepage = db.Column(db.String(200))

    genres = db.relationship('Movie_Genre', back_populates='movie')
    keywords = db.relationship('Movie_Keywords', back_populates='movie')
    production_countries = db.relationship('Movie_Production_Countries', back_populates='movie')
    spoken_languages = db.relationship('Movie_Spoken_Languages', back_populates='movie')
    cast = db.relationship('Movies_Cast', back_populates='movie')

class Movies_Cast(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movies.id'), primary_key=True)
    actor_id = db.Column(db.Integer, db.ForeignKey('cast.actor_id'), primary_key=True)
    movie = db.relationship('Movies', back_populates='cast')
    cast_member = db.relationship('Cast', back_populates='movies')

class Production_Countries(db.Model):
    country_id = db.Column(db.Integer, primary_key=True)
    country_name = db.Column(db.String(100))
    iso_code = db.Column(db.String(10))

    movies = db.relationship('Movie_Production_Countries', back_populates='country')

class Spoken_Languages(db.Model):
    language_id = db.Column(db.Integer, primary_key=True)
    language_name = db.Column(db.String(100))
    iso_code = db.Column(db.String(10))

    movies = db.relationship('Movie_Spoken_Languages', back_populates='language')


@app.route('/casts', methods=['GET'])
def get_casts():
    casts = Cast.query.all()
    return jsonify([{'actor_id': cast.actor_id, 'name': cast.name, 'gender': cast.gender} for cast in casts])

@app.route('/cast/<int:actor_id>', methods=['GET'])
def get_cast(actor_id):
    cast = Cast.query.get_or_404(actor_id)
    return jsonify({'actor_id': cast.actor_id, 'name': cast.name, 'gender': cast.gender})

@app.route('/genres', methods=['GET'])
def get_genres():
    genres = Genres.query.all()
    return jsonify([{'genre_id': genre.genre_id, 'genre_name': genre.genre_name} for genre in genres])

@app.route('/genre/<int:genre_id>', methods=['GET'])
def get_genre(genre_id):
    genre = Genres.query.get_or_404(genre_id)
    return jsonify({'genre_id': genre.genre_id, 'genre_name': genre.genre_name})

@app.route('/keywords', methods=['GET'])
def get_keywords():
    keywords = Keywords.query.all()
    return jsonify([{'keyword_id': keyword.keyword_id, 'keyword_name': keyword.keyword_name} for keyword in keywords])

@app.route('/keyword/<int:keyword_id>', methods=['GET'])
def get_keyword(keyword_id):
    keyword = Keywords.query.get_or_404(keyword_id)
    return jsonify({'keyword_id': keyword.keyword_id, 'keyword_name': keyword.keyword_name})

@app.route('/movies', methods=['GET'])
def get_movies():
    movies = Movies.query.all()
    return jsonify([{
        'id': movie.id, 'title': movie.title, 'overview': movie.overview, 'release_date': movie.release_date
    } for movie in movies])

@app.route('/movie/<int:movie_id>', methods=['GET'])
def get_movie(movie_id):
    movie = Movies.query.get_or_404(movie_id)
    return jsonify({
        'id': movie.id, 'title': movie.title, 'original_title': movie.original_title, 'overview': movie.overview,
        'budget': movie.budget, 'revenue': movie.revenue, 'release_date': movie.release_date, 'runtime': movie.runtime,
        'status': movie.status, 'tagline': movie.tagline, 'popularity': movie.popularity, 'vote_average': movie.vote_average,
        'vote_count': movie.vote_count, 'original_language': movie.original_language, 'homepage': movie.homepage
    })

@app.route('/movie/<int:movie_id>/genres', methods=['GET'])
def get_movie_genres(movie_id):
    movie = Movies.query.get_or_404(movie_id)
    genres = [genre.genre.genre_name for genre in movie.genres]
    return jsonify({'movie_id': movie.id, 'genres': genres})

@app.route('/movie/<int:movie_id>/keywords', methods=['GET'])
def get_movie_keywords(movie_id):
    movie = Movies.query.get_or_404(movie_id)
    keywords = [keyword.keyword.keyword_name for keyword in movie.keywords]
    return jsonify({'movie_id': movie.id, 'keywords': keywords})

@app.route('/movie/<int:movie_id>/production_countries', methods=['GET'])
def get_movie_production_countries(movie_id):
    movie = Movies.query.get_or_404(movie_id)
    countries = [country.country.country_name for country in movie.production_countries]
    return jsonify({'movie_id': movie.id, 'production_countries': countries})

@app.route('/movie/<int:movie_id>/spoken_languages', methods=['GET'])
def get_movie_spoken_languages(movie_id):
    movie = Movies.query.get_or_404(movie_id)
    languages = [language.language.language_name for language in movie.spoken_languages]
    return jsonify({'movie_id': movie.id, 'spoken_languages': languages})

@app.route('/movie/<int:movie_id>/cast', methods=['GET'])
def get_movie_cast(movie_id):
    movie = Movies.query.get_or_404(movie_id)
    cast = [{'actor_id': actor.actor_id, 'name': actor.name} for actor in movie.cast]
    return jsonify({'movie_id': movie.id, 'cast': cast})




from werkzeug.serving import run_simple
run_simple("localhost", 5000, app)

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Mar/2025 12:16:36] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Mar/2025 12:16:38] "GET /favicon.ico HTTP/1.1" 404 -
[2025-03-27 12:16:45,753] ERROR in app: Exception on /movies [GET]
Traceback (most recent call last):
  File "c:\Users\Balmee Hunumunt\OneDrive\Desktop\cinemind\.venv\Lib\site-packages\sqlalchemy\orm\mapper.py", line 2522, in get_property
    return self._props[key]
           ~~~~~~~~~~~^^^^^
KeyError: 'movies'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Balmee Hunumunt\OneDrive\Desktop\cinemind\.venv\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Balmee Hunumunt\OneDrive\Desktop\cinemind\.venv\Lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^

In [3]:
%tb

SystemExit: 1

Reduced class and endpoint for testing, server ran but json printed {"error":"Internal Server Error"}

In [3]:
from flask import Flask, jsonify
from flask_sqlalchemy import SQLAlchemy



app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///cinemind.db'
db.init_app(app)
db = SQLAlchemy()

# Define Models
class Movie(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String, nullable=False)
    overview = db.Column(db.String)
    release_date = db.Column(db.String)
    runtime = db.Column(db.Integer)

class Genre(db.Model):
    genre_id = db.Column(db.Integer, primary_key=True)
    genre_name = db.Column(db.String, nullable=False)

class MovieGenre(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movie.id'), primary_key=True)
    genre_id = db.Column(db.Integer, db.ForeignKey('genre.genre_id'), primary_key=True)

class Cast(db.Model):
    actor_id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String, nullable=False)
    gender = db.Column(db.Integer)

class MovieCast(db.Model):
    movie_id = db.Column(db.Integer, db.ForeignKey('movie.id'), primary_key=True)
    actor_id = db.Column(db.Integer, db.ForeignKey('cast.actor_id'), primary_key=True)
    character = db.Column(db.String)

# API Endpoints
import traceback

@app.route('/movies', methods=['GET'])
def get_movies():
    try:
        movies = Movie.query.all()
        return jsonify([{'id': m.id, 'title': m.title, 'overview': m.overview} for m in movies])
    except Exception as e:
        print("Error:", str(e))
        traceback.print_exc()  # Print full error traceback
        return jsonify({"error": "Internal Server Error"}), 500


@app.route('/movies/<int:id>', methods=['GET'])
def get_movie(id):
    movie = Movie.query.get_or_404(id)
    return jsonify({'id': movie.id, 'title': movie.title, 'overview': movie.overview})

@app.route('/genres', methods=['GET'])
def get_genres():
    genres = Genre.query.all()
    return jsonify([{'id': g.genre_id, 'name': g.genre_name} for g in genres])

@app.route('/movies/<int:id>/cast', methods=['GET'])
def get_movie_cast(id):
    cast = db.session.query(Cast.name, MovieCast.character).join(MovieCast, Cast.actor_id == MovieCast.actor_id).filter(MovieCast.movie_id == id).all()
    return jsonify([{'actor': c[0], 'character': c[1]} for c in cast])

from werkzeug.serving import run_simple
run_simple("localhost", 5000, app)

 * Running on http://localhost:5000
Press CTRL+C to quit


Testing to see if table is imported correctle/exists - which shows it doesnt

In [6]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///cinemind.db'
db = SQLAlchemy(app)

with app.app_context():
    try:
        result = db.session.execute(text("SELECT * FROM Movie LIMIT 5;")).fetchall()
        print(result)
    except Exception as e:
        print("Database Error:", str(e))

Database Error: (sqlite3.OperationalError) no such table: Movie
[SQL: SELECT * FROM Movie LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


final test to show if database is connected properly - failed

In [7]:
import os

db_path = os.path.abspath("cinemind.db")
print("Database path:", db_path)
print("Database exists:", os.path.exists(db_path))

Database path: c:\Users\Balmee Hunumunt\OneDrive\Desktop\cinemind\backend\app\cinemind.db
Database exists: False


Produced a working server, however does not have any data shown in json

In [2]:
from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///cinemind.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)


# Initialize empty lists for entities, or use these to interact with the database
casts = []
genres = []
keywords = []
movies = []
movie_genres = []
movie_keywords = []
movie_production_countries = []
movie_spoken_languages = []

# Model classes
class Cast(db.Model):
    __tablename__ = 'casts'
    actor_id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100))
    gender = db.Column(db.String(10))

class Genres(db.Model):
    __tablename__ = 'genres'
    genre_id = db.Column(db.Integer, primary_key=True)
    genre_name = db.Column(db.String(100))

class Keywords(db.Model):
    __tablename__ = 'keywords'
    keyword_id = db.Column(db.Integer, primary_key=True)
    keyword_name = db.Column(db.String(100))

class Movies(db.Model):
    __tablename__ = 'movies'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200))
    original_title = db.Column(db.String(200))
    overview = db.Column(db.String(1000))
    budget = db.Column(db.Integer)
    revenue = db.Column(db.Integer)
    release_date = db.Column(db.String(20))
    runtime = db.Column(db.Integer)
    status = db.Column(db.String(50))
    tagline = db.Column(db.String(200))
    popularity = db.Column(db.Float)
    vote_average = db.Column(db.Float)
    vote_count = db.Column(db.Integer)
    original_language = db.Column(db.String(10))
    homepage = db.Column(db.String(200))

# ----- Cast Endpoints -----
@app.route('/casts', methods=['GET'])
def get_casts():
    casts = Cast.query.all()
    return jsonify([cast.__dict__ for cast in casts])

@app.route('/casts/<int:actor_id>', methods=['GET'])
def get_cast(actor_id):
    cast = Cast.query.get(actor_id)
    if cast is None:
        return jsonify({"error": "Cast not found"}), 404
    return jsonify(cast.__dict__)

# ----- Genre Endpoints -----
@app.route('/genres', methods=['GET'])
def get_genres():
    genres = Genres.query.all()
    return jsonify([genre.__dict__ for genre in genres])

@app.route('/genres/<int:genre_id>', methods=['GET'])
def get_genre(genre_id):
    genre = Genres.query.get(genre_id)
    if genre is None:
        return jsonify({"error": "Genre not found"}), 404
    return jsonify(genre.__dict__)

# ----- Movie Endpoints -----
@app.route('/movies', methods=['GET'])
def get_movies():
    movies = Movies.query.all()
    return jsonify([movie.__dict__ for movie in movies])

@app.route('/api/movies/<int:id>', methods=['GET'])
def get_movie(id):
    movie = Movies.query.get(id)
    if movie is None:
        return jsonify({"error": "Movie not found"}), 404
    return jsonify(movie.__dict__)


if __name__ == '__main__':
    with app.app_context():
        db.create_all()  # Create tables in the database
    try:
        app.run(debug=False, use_reloader=False)
    except SystemExit:
        pass  # Handle SystemExit to avoid interrupting the kernel


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Added a character limit to strings so not to overload server

In [ ]:
import requests

# Test to get a single cast member
response = requests.get('http://127.0.0.1:5000/cast/1')
print("GET /cast/1 Response:", response.json())

# Test to get all cast members
response = requests.get('http://127.0.0.1:5000/casts')
print("GET /casts Response:", response.json())

# Test to get a single genre
response = requests.get('http://127.0.0.1:5000/genre/1')
print("GET /genre/1 Response:", response.json())

# Test to get all genres
response = requests.get('http://127.0.0.1:5000/genres')
print("GET /genres Response:", response.json())

# Test to get a single movie
response = requests.get('http://127.0.0.1:5000/movie/1')
print("GET /movie/1 Response:", response.json())

# Test to get all movies
response = requests.get('http://127.0.0.1:5000/movies')
print("GET /movies Response:", response.json())


Testing cell